### Ridge Reg Linear assignment

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import  scipy.sparse as ss
import sklearn.preprocessing as skp
import matplotlib.pyplot as plt

In [ ]:
data_train = pd.read_csv("./salary-train.csv")

In [ ]:
data_test = pd.read_csv("./salary-test-mini.csv")

In [ ]:
import re
def text_preprocess(t: str) -> str:
    z = t.lower()
    z = re.sub('[^a-zA-Z0-9]', ' ', z)
    return z

df_train = data_train.assign(FullDescription = data_train['FullDescription'].apply(text_preprocess))\
           .fillna({'LocationNormalized': 'nan', 'ContractTime': 'nan'})

df_test = data_test.assign(FullDescription = data_test['FullDescription'].apply(text_preprocess))\
          .fillna({'LocationNormalized': 'nan', 'ContractTime': 'nan'})

In [ ]:
df_test.head()

In [ ]:
tfidf = TfidfVectorizer(min_df=5, max_df = 1e10).fit(df_train['FullDescription'])

X_tr_1 = tfidf.transform(df_train['FullDescription'])
X_te_1 = tfidf.transform(df_test['FullDescription'])

In [ ]:
oh = DictVectorizer().fit(df_train[['LocationNormalized', 'ContractTime']].to_dict('records'))

X_tr_2 = oh.transform(df_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_te_2 = oh.transform(df_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [ ]:
X_tr = ss.hstack([X_tr_1,X_tr_2 ])
X_te = ss.hstack([X_te_1, X_te_2])

In [ ]:
y_tr = df_train['SalaryNormalized']
y_te = df_test['SalaryNormalized']

In [ ]:
import sklearn.linear_model as sklm

In [ ]:
cl = sklm.Ridge(alpha = 1, random_state=241)
cl.fit(X_tr, y_tr)

In [ ]:
with open("t1.txt", "w") as fh:
    s = " ".join(["%.2f" % z for z in list(cl.predict(X_te))])
    fh.write(s)

### PCA Assignment 

In [ ]:
from sklearn.decomposition import PCA
import numpy as np

In [ ]:
df = pd.read_csv("close_prices.csv")
dj = pd.read_csv("djia_index.csv")
dj = dj.set_index(pd.DatetimeIndex(dj.date)).drop(columns=['date'])
X = df.set_index(pd.DatetimeIndex(df.date)).drop(columns=['date'])

In [ ]:
mpca = PCA(n_components=10).fit(X)

In [ ]:
n = range(1, len(X.columns) + 1)
d  = [PCA(n_components=k).fit(X).explained_variance_ratio_.cumsum()[-1] for k in n]
d1 = [PCA(n_components=k).fit(skp.StandardScaler().fit_transform(X)).explained_variance_ratio_.cumsum()[-1] for k in n]

In [ ]:
plt.plot(n, d, label='no std')
plt.plot(n, d1, label='std')
plt.show()